In [27]:
import os
import pandas as pd

In [28]:
# FUNCIONES REUTILIZABLES

def cargar_datos(nombre_archivo):
    ruta = os.path.join("./data", nombre_archivo)
    return pd.read_excel(ruta)

In [29]:
def limpiar(df):
    df = df.dropna().copy()
    df.columns = df.columns.str.strip()
    for col in df.select_dtypes(include="object").columns:
        df[col] = df[col].str.strip()
    if "AÑO" in df.columns:
        df["AÑO"] = df["AÑO"].astype(int)
    return df

In [30]:
def transformar(df, columna_valor_original, nombre_columna_nueva):
    columnas_renombrar = {
        "AÑO": "Año",
        "CCAA_DESTINO": "CCAA",
        columna_valor_original: nombre_columna_nueva}
    df = df.rename(columns={k: v for k, v in columnas_renombrar.items() if k in df.columns})
    return df.groupby(["Año", "CCAA"], as_index=False)[nombre_columna_nueva].sum()

In [31]:
# USO CON MIS DATASETS
# 1-CARGAR

df_frontur = cargar_datos("FRONTUR_DL.xlsx")
df_nacional = cargar_datos("TURISMO_INTERNO_PROV_CCAA_DL.xlsx")
df_gasto = cargar_datos("GASTO_TPV_DESTINO_CCAA_MES_DL.xlsx")

In [32]:
# 2-LIMPIAR

df_frontur = limpiar(df_frontur)
df_nacional = limpiar(df_nacional)
df_gasto = limpiar(df_gasto)

In [33]:
# 3-TRANSFORMAR

df_frontur_transformado = transformar(df_frontur, "VISITANTES", "Turistas_Internacionales")
df_nacional_transformado = transformar(df_nacional, "TURISTAS", "Turistas_Nacionales")
df_gasto_transformado = transformar(df_gasto, "GASTO", "Gasto")

In [34]:
# 4-UNIR

df_union_internacional_gasto = pd.merge(df_frontur_transformado, df_gasto_transformado, on=["Año", "CCAA"], how="inner")
df_union_nacional_gasto = pd.merge(df_nacional_transformado, df_gasto_transformado, on=["Año", "CCAA"], how="inner")

In [50]:
# IMPRIMIR SOLUCIONES QUE SE QUIERAN VER EN PANTALLA

print("COLUMNAS FRONTUR:", df_frontur.columns)
print("COLUMNAS NACIONAL:", df_nacional.columns)
print("COLUMNAS GASTO:", df_gasto.columns)

COLUMNAS FRONTUR: Index(['Unnamed: 0', 'AÑO', 'MES', 'PAIS_RESIDENCIA', 'TIPO_VISITANTE',
       'CCAA_DESTINO', 'VISITANTES'],
      dtype='object')
COLUMNAS NACIONAL: Index(['Unnamed: 0', 'AÑO', 'MES', 'CCAA_ORIGEN', 'PROVINCIA_ORIGEN',
       'CCAA_DESTINO', 'PROVINCIA_DESTINO', 'TURISTAS', 'PERNOCTACIONES',
       'ESTANCIA_MEDIA'],
      dtype='object')
COLUMNAS GASTO: Index(['Unnamed: 0', 'AÑO', 'MES', 'CCAA_DESTINO', 'TIPO_ORIGEN', 'GASTO',
       'TRANSACCION'],
      dtype='object')


In [51]:
print("PRIMERAS FILAS DE FRONTUR:")
print(df_frontur.head(2))

print("PRIMERAS FILAS DE NACIONAL:")
print(df_nacional.head(2))

print("PRIMERAS FILAS DE GASTO:")
print(df_gasto.head(2))

PRIMERAS FILAS DE FRONTUR:
   Unnamed: 0   AÑO  MES PAIS_RESIDENCIA TIPO_VISITANTE CCAA_DESTINO  \
0           0  2018    1        Alemania  Excursionista    Andalucía   
1           1  2018    1        Alemania  Excursionista     Baleares   

   VISITANTES  
0         818  
1        3694  
PRIMERAS FILAS DE NACIONAL:
   Unnamed: 0   AÑO  MES CCAA_ORIGEN PROVINCIA_ORIGEN CCAA_DESTINO  \
0           0  2019    7   Andalucía          Almería    Andalucía   
1           1  2019    7   Andalucía          Almería    Andalucía   

  PROVINCIA_DESTINO TURISTAS PERNOCTACIONES ESTANCIA_MEDIA  
0             Cádiz   8910,0        33536,0            3,8  
1           Córdoba   3468,0        11981,0            3,5  
PRIMERAS FILAS DE GASTO:
   Unnamed: 0   AÑO  MES CCAA_DESTINO    TIPO_ORIGEN        GASTO  TRANSACCION
0           0  2019    1    Andalucía  Internacional  18481196,78       414192
1           1  2019    1    Andalucía  Interregional  34039254,92       952493


In [52]:
print("FILAS EN FRONTUR TRANSFORMADO:", len(df_frontur_transformado))
print("FILAS EN GASTO TRANSFORMADO:", len(df_gasto_transformado))
print("FILAS EN UNIÓN + GASTO:", len(df_union_internacional_gasto))

FILAS EN FRONTUR TRANSFORMADO: 155
FILAS EN GASTO TRANSFORMADO: 133
FILAS EN UNIÓN + GASTO: 94


In [53]:
print("TURISMO INTERNACIONAL TRANSFORMADO:")
print(df_frontur_transformado.head(2))

print("TURISMO NACIONAL TRANSFORMADO:")
print(df_nacional_transformado.head(2))

print("GASTO TRANSFORMADO:")
print(df_gasto_transformado.head(2))

TURISMO INTERNACIONAL TRANSFORMADO:
    Año       CCAA  Turistas_Internacionales
0  2018  Andalucía                  14017705
1  2018     Aragón                    868433
TURISMO NACIONAL TRANSFORMADO:
    Año       CCAA                                Turistas_Nacionales
0  2019  Andalucía  8910,03468,049626,02801,09247,020363,06920,083...
1  2019     Aragón  666,0515,0975,0513,0303,01139,0614,0478,01326,...
GASTO TRANSFORMADO:
    Año       CCAA                                              Gasto
0  2019  Andalucía  18481196,7834039254,9233721507,9667760762,8818...
1  2019     Aragón  503190,6810511516,071976147,6112487663,6852437...


In [56]:
print("INTERNACIONAL + GASTO:")
print(df_union_internacional_gasto.head(2))

print("NACIONAL + GASTO:")
print(df_union_nacional_gasto.head(2))

INTERNACIONAL + GASTO:
    Año       CCAA  Turistas_Internacionales  \
0  2019  Andalucía                  14652617   
1  2019     Aragón                    853190   

                                               Gasto  
0  18481196,7834039254,9233721507,9667760762,8818...  
1  503190,6810511516,071976147,6112487663,6852437...  
NACIONAL + GASTO:
    Año       CCAA                                Turistas_Nacionales  \
0  2019  Andalucía  8910,03468,049626,02801,09247,020363,06920,083...   
1  2019     Aragón  666,0515,0975,0513,0303,01139,0614,0478,01326,...   

                                               Gasto  
0  18481196,7834039254,9233721507,9667760762,8818...  
1  503190,6810511516,071976147,6112487663,6852437...  
